In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math


Feature Map.

In [ ]:
num_qubits = 4
num_layers = 2

dev = qml.device("default.qubit", wires=num_qubits)
# quantum circuit functions
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

Ansatz modificado a StronglyEntanglingLayers

In [ ]:
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)
    qml.StronglyEntanglingLayers(weights=weights, wires=range(4), ranges=[2, 3], imprimitive=qml.ops.CZ)
    return qml.expval(qml.PauliZ(0))

Salida del circuito cuántico sumado al sesgo.

In [ ]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

Definición de la función de pérdida.

In [ ]:

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

Definición de la función de evaluación de desempeño.

In [ ]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

Definición de la función de evaluación de accuracy.

In [ ]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

Definición de la función de costo.

In [ ]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [ ]:
import zipfile

with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
    zip_ref.extractall('titanic_data')

# Verifica los archivos extraídos
import os

print("Archivos extraídos:")
print(os.listdir('titanic_data'))


Archivos extraídos:
['train.csv', 'gender_submission.csv', 'test.csv']


Preprocesamiento de los datos.

In [ ]:
# preparaing data
df_train = pd.read_csv('/content/titanic_data/train.csv')

df_train['Pclass'] = df_train['Pclass'].astype(str)

df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)

# I will fill missings with the median
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']
# En este bloque se hace la separaiación entre datos de prueba y datos de test.
X_train, X_test, y_train, y_test = train_test_split(df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived'])

X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)


Inicialización de los parámetros.

In [ ]:
np.random.seed(0)
shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
weights_init = np.random.random(size=shape)
bias_init = np.array(0.0, requires_grad=True)

opt = GradientDescentOptimizer(0.3)
num_it = 20
batch_size = math.floor(len(X_train)/num_it)

weights = weights_init
bias = bias_init

In [ ]:
print("Los pesos iniciales son: ", weights_init)
print("Los bias iniciales son:",bias_init)

Los pesos iniciales son:  [[[0.5488135  0.71518937 0.60276338]
  [0.54488318 0.4236548  0.64589411]
  [0.43758721 0.891773   0.96366276]
  [0.38344152 0.79172504 0.52889492]]

 [[0.56804456 0.92559664 0.07103606]
  [0.0871293  0.0202184  0.83261985]
  [0.77815675 0.87001215 0.97861834]
  [0.79915856 0.46147936 0.78052918]]]
Los bias iniciales son: 0.0


Entrenamiento del circuito

In [ ]:
for it in range(num_it):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )

Iter:     1 | Cost: 1.0178197 | Accuracy: 0.6167291 
Iter:     2 | Cost: 0.9655172 | Accuracy: 0.6167291 
Iter:     3 | Cost: 0.9707957 | Accuracy: 0.6167291 
Iter:     4 | Cost: 0.9451315 | Accuracy: 0.6167291 
Iter:     5 | Cost: 0.9355544 | Accuracy: 0.6167291 
Iter:     6 | Cost: 0.9597053 | Accuracy: 0.6304619 
Iter:     7 | Cost: 1.0365776 | Accuracy: 0.6304619 
Iter:     8 | Cost: 1.0920459 | Accuracy: 0.3695381 
Iter:     9 | Cost: 0.9550147 | Accuracy: 0.6304619 
Iter:    10 | Cost: 0.9412038 | Accuracy: 0.6167291 
Iter:    11 | Cost: 0.9416059 | Accuracy: 0.6304619 
Iter:    12 | Cost: 1.0280253 | Accuracy: 0.6304619 
Iter:    13 | Cost: 0.9898322 | Accuracy: 0.6304619 
Iter:    14 | Cost: 0.9506577 | Accuracy: 0.6167291 
Iter:    15 | Cost: 0.9673344 | Accuracy: 0.6304619 
Iter:    16 | Cost: 0.9336502 | Accuracy: 0.6304619 
Iter:    17 | Cost: 0.9890687 | Accuracy: 0.6304619 
Iter:    18 | Cost: 0.9627048 | Accuracy: 0.6167291 
Iter:    19 | Cost: 0.9934616 | Accuracy: 0.63

organización de los datos de test.

In [ ]:
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

Predición del circuito

In [ ]:
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]


Pesos entrenados y bias.

In [ ]:
print("Los pesos entrenados son: ",weights)
print("Los pesos entrenados son",bias)

Los pesos entrenados son:  [[[0.5488135  0.91709821 0.4526575 ]
  [0.54488318 0.4236548  0.64589411]
  [0.43758721 0.891773   0.96366276]
  [0.38344152 0.79172504 0.52889492]]

 [[0.41793869 1.20018895 0.07103606]
  [0.0871293  0.0202184  0.83261985]
  [0.77815675 0.87001215 0.97861834]
  [0.79915856 0.46147936 0.78052918]]]
Los pesos entrenados son -0.25896367305103657


Métricas de presición del sistema.

In [ ]:
accuracy_score(Y_test, predictions)
precision_score(Y_test, predictions)
recall_score(Y_test, predictions)
f1_score(Y_test, predictions, average='macro')

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.5057471264367815

Al compar la eficiencia obtenida con el cricuito orginal, es evidente que no hubo una buena respuesta al entrenamiento por parte del presente circuito( el modificado), puesto que a diferencia de la primera parte donde el circuito orginal tuvo un 0.77, ahora, solo alcanzó un 0.51 aproximadamente. Esto muestra que en términos de predicción la primera implementación tiene una mayor fidelidad. No obstante, cabe resaltar que este se entrenó por menos épocas lo cual tiene una alta influencia en la respuesta y además, la arquitectura puede no ser idónea para este tipo de aplicaciones.